In [ ]:
!pip install tensorflow-text

In [2]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_text as text
from keras import layers
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Arabic Dataset/01_firstExam_data_arabic_extra_gender_feature_v1.csv')

In [4]:
df.head()

,desk_no,name,school_name_arabic,administration,government_Arabic,branch,Percentage,student_type,status,arabic,...,chemistry,biology,geology,applied_math,physics,total,religion,altarbiat_alwatania,economics_statistics,gender
0,102300.0,احمد محمد احمد محمود ابوالقاسم,مدارس النيل المصريه بنين,أكتوبر,الجيزة,غير محدد,0%,نوعيات أخرى,ناجح,غير مقرر,...,غير مقرر,غير مقرر,غير مقرر,غير مقرر,غير مقرر,0.0,23,20,غير مقرر,M
1,102301.0,احمد محمد عبدالصادق سلامه,مدارس النيل المصريه بنين,أكتوبر,الجيزة,غير محدد,0%,نوعيات أخرى,ناجح,غير مقرر,...,غير مقرر,غير مقرر,غير مقرر,غير مقرر,غير مقرر,0.0,23,23,غير مقرر,M
2,102302.0,ادهم اشرف عبدالهادي معاذ,مدارس النيل المصريه بنين,أكتوبر,الجيزة,غير محدد,0%,نوعيات أخرى,ناجح,غير مقرر,...,غير مقرر,غير مقرر,غير مقرر,غير مقرر,غير مقرر,0.0,17,14,غير مقرر,M
3,102303.0,بيير جون نبيل غالي سعد,مدارس النيل المصريه بنين,أكتوبر,الجيزة,غير محدد,0%,نوعيات أخرى,ناجح,غير مقرر,...,غير مقرر,غير مقرر,غير مقرر,غير مقرر,غير مقرر,0.0,21,22,غير مقرر,M
4,102304.0,رامي محمود عبدالمنعم شحاته,مدارس النيل المصريه بنين,أكتوبر,الجيزة,غير محدد,0%,نوعيات أخرى,ناجح,غير مقرر,...,غير مقرر,غير مقرر,غير مقرر,غير مقرر,غير مقرر,0.0,19,21,غير مقرر,M


In [5]:
df.shape

(698196, 27)

In [6]:
df.drop(['desk_no','school_name_arabic','administration','government_Arabic','branch','Percentage',
         'student_type','status','arabic','first_foreign_lang','second_foreign_lang','pure_mathematics',
         'history','geography','philosophy','psychology','chemistry','biology','geology','applied_math',
         'physics','total','religion','altarbiat_alwatania','economics_statistics','gender'],
        axis=1, inplace= True)

In [7]:
df.head(10)

,name
0,احمد محمد احمد محمود ابوالقاسم
1,احمد محمد عبدالصادق سلامه
2,ادهم اشرف عبدالهادي معاذ
3,بيير جون نبيل غالي سعد
4,رامي محمود عبدالمنعم شحاته
5,زياد عزمي احمد محمد الشهاوي
6,عبدالرحمن وليد عصفور احمد مشعل
7,عبدالله مصطفي محمود عبدالحميد محمود
8,عمرو خالد محمود كامل السيد
9,كريم احمد محمد حامد


In [8]:
df['name'] = df['name'].str.split().str[:3].str.join(sep=" ")

In [9]:
df.head()

,name
0,احمد محمد احمد
1,احمد محمد عبدالصادق
2,ادهم اشرف عبدالهادي
3,بيير جون نبيل
4,رامي محمود عبدالمنعم


In [10]:
n  = int(len(df)*.3)
for index, row in df[:n].iterrows():
    df.loc[index][0] = ''.join(random.sample(df.iloc[index][0], len(df.iloc[index][0])))

In [11]:
df['type'] = 1 # Real
for index, row in df[:n].iterrows():
  df['type'][:n] = 0 # Fake

<ipython-input-11-2ffbaa3d14e7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type'][:n] = 0 # Fake


In [12]:
df = df.sample(frac=1).reset_index()

In [13]:
df.drop(['index'], axis=1, inplace= True)

In [14]:
df.head(10)

,name,type
0,ميبم وداديدحمز لوامح,0
1,بامشرر يسيغس,0
2,صابرين فتحي عثمان,1
3,ابراهيم حماده ابراهيم,1
4,رحاب علي شوقي,1
5,لحعيارد لباععامبددمينجط,0
6,خالد غريب عرابي,1
7,مهاب محمد دسوقي,1
8,احمد فارس عبدالرازق,1
9,بقطر ممدوح شحاته,1


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['name'], df['type'], test_size=.2, random_state = 1)

In [26]:
X_train.shape

(558556,)

In [27]:
X_test.shape

(139640,)

In [16]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(X_train.astype("str"))

train = tokenizer.texts_to_sequences(X_train.astype("str"))
test = tokenizer.texts_to_sequences(X_test.astype("str"))

vocab_size = len(tokenizer.word_index) + 1 

In [17]:
maxlen = 5

train = pad_sequences(train, padding = 'post', maxlen = maxlen)
test = pad_sequences(test, padding = 'post', maxlen = maxlen)

print(train[0, :])

[1787  740    0    0    0]


In [18]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [19]:
embedding_dim = 5

model = Sequential()
model.add(layers.Embedding(input_dim = vocab_size, output_dim = embedding_dim, input_length = maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 5)              1354960   
                                                                 
 flatten (Flatten)           (None, 25)                0         
                                                                 
 dense (Dense)               (None, 10)                260       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,355,231
Trainable params: 1,355,231
Non-trainable params: 0
_________________________________________________________________


In [20]:
history = model.fit(train, y_train, epochs = 5, verbose = 1, batch_size = 64)

Epoch 1/5
8728/8728 [==============================] - 41s 4ms/step - loss: 0.0178 - accuracy: 0.9940
Epoch 2/5
8728/8728 [==============================] - 32s 4ms/step - loss: 0.0032 - accuracy: 0.9991
Epoch 3/5
8728/8728 [==============================] - 30s 3ms/step - loss: 0.0028 - accuracy: 0.9992
Epoch 4/5
8728/8728 [==============================] - 30s 3ms/step - loss: 0.0026 - accuracy: 0.9993
Epoch 5/5
8728/8728 [==============================] - 38s 4ms/step - loss: 0.0025 - accuracy: 0.9993


In [21]:
loss, accuracy = model.evaluate(train, y_train, verbose = 1)
print("Training Accuracy: {:.4f}".format(accuracy))

loss, accuracy = model.evaluate(test, y_test, verbose = 1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

17455/17455 [==============================] - 37s 2ms/step - loss: 0.0023 - accuracy: 0.9994
Training Accuracy: 0.9994
4364/4364 [==============================] - 9s 2ms/step - loss: 0.0030 - accuracy: 0.9992
Testing Accuracy:  0.9992


In [22]:
inputs = ['احمد محمد محمود' ,'لهغالا بررهممتاا طط' ,'عمر اسماعيل السيد']
sequence = tokenizer.texts_to_sequences(inputs)
sequence = pad_sequences(sequence, padding = 'post', maxlen = maxlen)

predictions = model.predict(sequence)
predictions = np.where(predictions > 0.5, 1, 0)
print(predictions)

1/1 [==============================] - 0s 80ms/step
[[1]
 [0]
 [1]]


In [23]:
model.save("model.h5")

In [24]:
mod = tf.keras.models.load_model('model.h5')

In [25]:
inputs = ['احمد محمد محمود' ,'لهغالا بررهممتاا طط' ,'عمر اسماعيل السيد']
sequence = tokenizer.texts_to_sequences(inputs)
sequence = pad_sequences(sequence, padding = 'post', maxlen = maxlen)

predictions = mod.predict(sequence)
predictions = np.where(predictions > 0.5, 1, 0)
print(predictions)

1/1 [==============================] - 0s 47ms/step
[[1]
 [0]
 [1]]
